In [43]:
import mlflow

from mlflow.tracking import MlflowClient
from mlflow.entities import ViewType

In [44]:
MLFLOW_TRACKING_URI = "sqlite:///backend.db"
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

## Search Experiments

In [45]:
client.search_experiments(view_type=1)

[<Experiment: artifact_location='/home/mk/Playground/my_contrib/prefect-practice-mlops/mlruns/1', creation_time=1689672078862, experiment_id='1', last_update_time=1689672078862, lifecycle_stage='active', name='prototype', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1689672040038, experiment_id='0', last_update_time=1689672040038, lifecycle_stage='active', name='Default', tags={}>]

## Choosing best model

In [63]:
experiment_id = "1"

In [64]:
best_run_info = client.search_runs(
    experiment_ids=experiment_id,
    run_view_type=ViewType.ACTIVE_ONLY,
    order_by=["metrics.f1_score DESC"],
    max_results=1
)[0]

In [65]:
print(f"Run id: {best_run_info.info.run_id}, f1-score: {best_run_info.data.metrics['f1_score']:.3f}, AUC: {best_run_info.data.metrics['auc']:.3f}")

Run id: fddc8bc2db554dd2bfad2a7e4e48e952, f1-score: 0.805, AUC: 0.981


In [66]:
best_run_id = best_run_info.info.run_id

In [67]:
best_run_id

'fddc8bc2db554dd2bfad2a7e4e48e952'

## Registering the models

In [72]:
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [73]:
model_name = "diabetes-classifier"

In [74]:
best_model_uri = f"runs:/{best_run_id}/model"

In [75]:
best_model_uri

'runs:/fddc8bc2db554dd2bfad2a7e4e48e952/model'

In [78]:
reg_data = mlflow.register_model(
    model_uri=best_model_uri,
    name=model_name

)

Successfully registered model 'diabetes-classifier'.
2023/07/18 16:23:54 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: diabetes-classifier, version 1
Created version '1' of model 'diabetes-classifier'.


## Model Stage Registering

In [80]:
stage = "production"

In [81]:
client.transition_model_version_stage(
    name = model_name,
    version = reg_data.version,
    stage = stage,
    archive_existing_versions=False

)

<ModelVersion: aliases=[], creation_timestamp=1689674034357, current_stage='Production', description=None, last_updated_timestamp=1689674127311, name='diabetes-classifier', run_id='fddc8bc2db554dd2bfad2a7e4e48e952', run_link=None, source='/home/mk/Playground/my_contrib/prefect-practice-mlops/mlruns/1/fddc8bc2db554dd2bfad2a7e4e48e952/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=1>

## Update model description

In [82]:
from datetime import datetime

In [85]:
date = datetime.today().date()

In [86]:
date

datetime.date(2023, 7, 18)

In [88]:
client.update_model_version(
    name=model_name,
    version=reg_data.version,
    description=f"The model version {reg_data.version} was transition to {stage} on {date}"
)

<ModelVersion: aliases=[], creation_timestamp=1689674034357, current_stage='Production', description='The model version 1 was transition to production on 2023-07-18', last_updated_timestamp=1689674268348, name='diabetes-classifier', run_id='fddc8bc2db554dd2bfad2a7e4e48e952', run_link=None, source='/home/mk/Playground/my_contrib/prefect-practice-mlops/mlruns/1/fddc8bc2db554dd2bfad2a7e4e48e952/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=1>